<a href="https://colab.research.google.com/github/KieranDingwall/Natural-Language-Processing-Coursework---Kieran-Dingwall/blob/main/NLP_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Coursework

By Kieran Dingwall - Student Number: 2208619

### Section 1 - Dataset Exploring and Pre Processing

In [ ]:
# Imports
import pandas as pd
import numpy as np
# import io

# Load Dataset
product_data = pd.read_csv("pricerunner_aggregate.csv")
print(product_data)

       Product ID                                      Product Title  \
0               1                    apple iphone 8 plus 64gb silver   
1               2                apple iphone 8 plus 64 gb spacegrau   
2               3  apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...   
3               4                apple iphone 8 plus 64gb space grey   
4               5  apple iphone 8 plus gold 5.5 64gb 4g unlocked ...   
...           ...                                                ...   
35306       47350  smeg fab28 60cm retro style right hand hinge f...   
35307       47351  smeg fab28 60cm retro style left hand hinge fr...   
35308       47352  smeg fab28 60cm retro style left hand hinge fr...   
35309       47355     candy 60cm built under larder fridge cru160nek   
35310       47358           neff k4316x7gb built under larder fridge   

        Merchant ID   Cluster ID             Cluster Label   Category ID  \
0                 1            1  Apple iPhone 8 Plus 64GB 

In [ ]:
# Remove unused columns from data
product_df = product_data.drop(product_data.columns[[0, 2, 3, 5]], axis=1)
print(product_df)

                                           Product Title  \
0                        apple iphone 8 plus 64gb silver   
1                    apple iphone 8 plus 64 gb spacegrau   
2      apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...   
3                    apple iphone 8 plus 64gb space grey   
4      apple iphone 8 plus gold 5.5 64gb 4g unlocked ...   
...                                                  ...   
35306  smeg fab28 60cm retro style right hand hinge f...   
35307  smeg fab28 60cm retro style left hand hinge fr...   
35308  smeg fab28 60cm retro style left hand hinge fr...   
35309     candy 60cm built under larder fridge cru160nek   
35310           neff k4316x7gb built under larder fridge   

                  Cluster Label  Category Label  
0      Apple iPhone 8 Plus 64GB   Mobile Phones  
1      Apple iPhone 8 Plus 64GB   Mobile Phones  
2      Apple iPhone 8 Plus 64GB   Mobile Phones  
3      Apple iPhone 8 Plus 64GB   Mobile Phones  
4      Apple iPhone 8 Plus 64

In [ ]:
# Convert the feature columns into a numpy array
x = product_df["Product Title"].to_numpy()
y = product_df[" Category Label"].to_numpy()

# Testing outputs
#print(x)
#print(y)

['apple iphone 8 plus 64gb silver' 'apple iphone 8 plus 64 gb spacegrau'
 'apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim free smartphone in gold'
 ... 'smeg fab28 60cm retro style left hand hinge fridge with icebox pink'
 'candy 60cm built under larder fridge cru160nek'
 'neff k4316x7gb built under larder fridge']
['Mobile Phones' 'Mobile Phones' 'Mobile Phones' ... 'Fridges' 'Fridges'
 'Fridges']


In [ ]:
# Checking Balance

print(product_df["Product Title"].value_counts())
print(product_df[" Category Label"].value_counts())

# Balance is good

Product Title
washing machine                                                     90
built in fully integrated dishwasher                                35
frost free fridge freezer                                           34
washer dryer                                                        24
american fridge freezer                                             22
                                                                    ..
apple iphone 8 plus 64gb silver unlocked                             1
apple iphone 8 plus 14 cm 5.5 64 gb 12 mp ios 11 silver              1
iphone 8 plus sim free water dust resistant 64gb silver by apple     1
iphone 8 plus 64gb space grey                                        1
neff k4316x7gb built under larder fridge                             1
Name: count, Length: 30993, dtype: int64
 Category Label
Fridge Freezers     5501
Mobile Phones       4081
Washing Machines    4044
CPUs                3862
Fridges             3584
TVs                 356

### Section 2 - Representation Learning

In [ ]:
# Representation Learner
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_base = TfidfVectorizer(max_features=2000).fit_transform(x)
print(tfidf_base)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 247182 stored elements and shape (35311, 2000)>
  Coords	Values
  (0, 563)	0.47562446686894067
  (0, 1143)	0.45481272898552627
  (0, 1482)	0.4733224289764213
  (0, 377)	0.46740012928169644
  (0, 1642)	0.35274600698475417
  (1, 563)	0.35608862117257495
  (1, 1143)	0.3405073725965799
  (1, 1482)	0.3543651406619306
  (1, 375)	0.41707667122083275
  (1, 974)	0.35714545937478775
  (1, 1698)	0.5746086853970792
  (2, 563)	0.3331251324733054
  (2, 1143)	0.31854868945503123
  (2, 1482)	0.3315127959950069
  (2, 377)	0.32736484523179377
  (2, 51)	0.49365156932521237
  (2, 1643)	0.25020523620439966
  (2, 933)	0.19964760536977605
  (2, 1663)	0.28509409641708544
  (2, 1114)	0.1862759239990133
  (2, 997)	0.3291729646162801
  (3, 563)	0.39548184226606475
  (3, 1143)	0.37817687792502896
  (3, 1482)	0.39356769728379937
  (3, 377)	0.3886433038665723
  :	:
  (35307, 1665)	0.2440131092564437
  (35307, 946)	0.156914887522846
  (35307, 1101)	0.4146

### Section 3 - Algorithms

In [ ]:
# Classifier
# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

def mlp(dataset_size, num_classes):
  input_shape = (dataset_size,)

  model = Sequential()
  model.add(Dense(128, input_shape=input_shape, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  return model

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
# SKLearn Imports
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC

acc_score = []

pipeline = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf_base', TfidfTransformer()),
  ('clf', SVC(kernel='linear', C=1))
])

kf = StratifiedKFold(n_splits=5)
for train, test in kf.split(x,y):
  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]

  pipeline.fit(x_train, y_train)
  predictions = pipeline.predict(x_test)
  acc = accuracy_score(predictions, y_test)
  acc_score.append(acc)

print("Accuracy:", np.mean(acc_score))

Accuracy: 0.9448893446059141


### Section 4 - Evaluation